In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.stats import linregress
from pprint import pprint
import calendar
from calendar import month_abbr
import datetime as dt

In [2]:
Weather_Path = "Clean Data/SAWeatherAll.csv"
Drought_Path = "Clean Data/dm_export_20160705_20220705.csv"
J17_Well_Path = "Clean Data/j17.csv"


Weather_Data_df = pd.read_csv(Weather_Path, encoding="utf-8")
Drought_Data_df = pd.read_csv(Drought_Path, encoding="utf-8")
J17_Well_Data_df = pd.read_csv(J17_Well_Path, encoding="utf-8")

In [3]:
Weather_Data_df

,Unnamed: 0,Month,Day,Year,Temp Max,Temp Avg,Temp Min,Humidity Max,Humidity Avg,Humidity Min,Precipitation
0,0,Jan,1,2016,49,46.3,44,73,61.5,51,0.00
1,1,Jan,2,2016,48,45.0,43,93,67.2,48,0.00
2,2,Jan,3,2016,61,47.7,35,96,75.6,49,0.22
3,3,Jan,4,2016,57,42.7,33,93,67.6,37,0.00
4,4,Jan,5,2016,57,44.9,30,96,74.4,55,0.00
...,...,...,...,...,...,...,...,...,...,...,...
2398,207,Jul,27,2022,101,86.5,74,93,61.2,28,0.00
2399,208,Jul,28,2022,100,86.2,76,94,63.1,31,0.00
2400,209,Jul,29,2022,100,86.8,76,94,63.0,31,0.00
2401,210,Jul,30,2022,100,87.6,73,96,56.7,27,0.00


In [4]:
#only run this ONCE you will get an error if you dont restart the kernel
#changes the months in month_abbr to lower case
lower_ma = [m.lower() for m in month_abbr]

#changes the months in df to lowercase and maps the values to numerical months
Weather_Data_df['Month'] = Weather_Data_df['Month'].str.lower().map(lambda m: lower_ma.index(m)).astype('Int8')


In [5]:
#uses the apply method to merge our numerical month values into one column
cols=["Month", "Day", "Year"]
Weather_Data_df['Date'] = Weather_Data_df[cols].apply(lambda x: '/'.join(x.values.astype(str)), axis="columns")

Weather_Data_df = Weather_Data_df.drop(columns = ["Month", "Day", "Year", "Unnamed: 0"])

Weather_Data_df["Date"] = pd.to_datetime(Weather_Data_df["Date"])

In [7]:
Weather_Data_df.dtypes

Temp Max                  int64
Temp Avg                float64
Temp Min                  int64
Humidity Max              int64
Humidity Avg            float64
Humidity Min              int64
Precipitation           float64
Date             datetime64[ns]
dtype: object

In [8]:
#we only need to filter for last 5 years
# double check and ensure its J17 only
#clean it 

J17_Well_Data_df = J17_Well_Data_df.dropna()
J17_Well_Data_df

,Site,DailyHighDate,WaterLevelElevation
0,J17WL,7/5/2022,635.98
1,J17WL,7/4/2022,636.85
2,J17WL,7/3/2022,637.15
3,J17WL,7/2/2022,636.71
4,J17WL,7/1/2022,636.55
...,...,...,...
2373,J17WL,1/5/2016,667.55
2374,J17WL,1/4/2016,667.70
2375,J17WL,1/3/2016,666.69
2376,J17WL,1/2/2016,666.37


In [9]:
J17_Well_Data_df = J17_Well_Data_df.dropna()
J17_Well_Data_df["DailyHighDate"]= pd.to_datetime(J17_Well_Data_df["DailyHighDate"])
J17_Well_Data_df.dtypes

Site                           object
DailyHighDate          datetime64[ns]
WaterLevelElevation           float64
dtype: object

In [10]:
#create bins for aquifer water level
waterlevel = [0, 639, 649, 659, 700]

# Create the names for what restriction the city will be placed under based on water level
restriction = ["restriction 3", " restriction 2", "restriction 1", "no restriction"]

#create a new column for restriction level, we can filter based off these later
J17_Well_Data_df["Restriction Level"] = pd.cut(J17_Well_Data_df["WaterLevelElevation"], waterlevel, labels=restriction)

In [11]:
J17_Well_Data_df

,Site,DailyHighDate,WaterLevelElevation,Restriction Level
0,J17WL,2022-07-05,635.98,restriction 3
1,J17WL,2022-07-04,636.85,restriction 3
2,J17WL,2022-07-03,637.15,restriction 3
3,J17WL,2022-07-02,636.71,restriction 3
4,J17WL,2022-07-01,636.55,restriction 3
...,...,...,...,...
2373,J17WL,2016-01-05,667.55,no restriction
2374,J17WL,2016-01-04,667.70,no restriction
2375,J17WL,2016-01-03,666.69,no restriction
2376,J17WL,2016-01-02,666.37,no restriction


In [12]:
#Change to datetime
Drought_Data_df[['ValidStart','ValidEnd']] = Drought_Data_df[['ValidStart','ValidEnd']].apply(pd.to_datetime)
Drought_Data_df.dtypes

MapDate                       int64
FIPS                          int64
County                       object
State                        object
None                        float64
D0                          float64
D1                          float64
D2                          float64
D3                          float64
D4                          float64
ValidStart           datetime64[ns]
ValidEnd             datetime64[ns]
StatisticFormatID             int64
dtype: object

In [13]:
Drought_Data_df

,MapDate,FIPS,County,State,None,D0,D1,D2,D3,D4,ValidStart,ValidEnd,StatisticFormatID
0,20220705,48029,Bexar County,TX,0.0,100.0,100.0,100.0,100.0,27.38,2022-07-05,2022-07-11,1
1,20220628,48029,Bexar County,TX,0.0,100.0,100.0,100.0,100.0,49.45,2022-06-28,2022-07-04,1
2,20220621,48029,Bexar County,TX,0.0,100.0,100.0,100.0,100.0,49.45,2022-06-21,2022-06-27,1
3,20220614,48029,Bexar County,TX,0.0,100.0,100.0,100.0,100.0,49.45,2022-06-14,2022-06-20,1
4,20220607,48029,Bexar County,TX,0.0,100.0,100.0,100.0,100.0,10.40,2022-06-07,2022-06-13,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,20160802,48029,Bexar County,TX,100.0,0.0,0.0,0.0,0.0,0.00,2016-08-02,2016-08-08,1
310,20160726,48029,Bexar County,TX,100.0,0.0,0.0,0.0,0.0,0.00,2016-07-26,2016-08-01,1
311,20160719,48029,Bexar County,TX,100.0,0.0,0.0,0.0,0.0,0.00,2016-07-19,2016-07-25,1
312,20160712,48029,Bexar County,TX,100.0,0.0,0.0,0.0,0.0,0.00,2016-07-12,2016-07-18,1


# Filtering Data

In [14]:
#for aquifer data the most recent date we have is 2022-07-05, filter accordingly to get 5 yrs worth of data
#we do this for all 3 data sets we have
Weather_Data_2016_df = Weather_Data_df.loc[(Weather_Data_df["Date"] >= "2016-07-05") & (Weather_Data_df["Date"] <= "2016-12-31")]
Weather_Data_2017_df = Weather_Data_df.loc[(Weather_Data_df["Date"] >= "2017-01-01") & (Weather_Data_df["Date"] <= "2017-12-31")]
Weather_Data_2018_df = Weather_Data_df.loc[(Weather_Data_df["Date"] >= "2018-01-01") & (Weather_Data_df["Date"] <= "2018-12-31")]
Weather_Data_2019_df = Weather_Data_df.loc[(Weather_Data_df["Date"] >= "2019-01-01") & (Weather_Data_df["Date"] <= "2019-12-31")]
Weather_Data_2020_df = Weather_Data_df.loc[(Weather_Data_df["Date"] >= "2020-01-01") & (Weather_Data_df["Date"] <= "2020-12-31")]
Weather_Data_2021_df = Weather_Data_df.loc[(Weather_Data_df["Date"] >= "2021-01-01") & (Weather_Data_df["Date"] <= "2021-12-31")]
Weather_Data_2022_df = Weather_Data_df.loc[(Weather_Data_df["Date"] >= "2022-01-01") & (Weather_Data_df["Date"] <= "2022-07-05")]

In [15]:
J17_2016_df = J17_Well_Data_df.loc[(J17_Well_Data_df["DailyHighDate"] >= "2016-07-05") & (J17_Well_Data_df["DailyHighDate"] <= "2016-12-31")]
J17_2017_df = J17_Well_Data_df.loc[(J17_Well_Data_df["DailyHighDate"] >= "2017-01-01") & (J17_Well_Data_df["DailyHighDate"] <= "2017-12-31")]
J17_2018_df = J17_Well_Data_df.loc[(J17_Well_Data_df["DailyHighDate"] >= "2018-01-01") & (J17_Well_Data_df["DailyHighDate"] <= "2018-12-31")]
J17_2019_df = J17_Well_Data_df.loc[(J17_Well_Data_df["DailyHighDate"] >= "2019-01-01") & (J17_Well_Data_df["DailyHighDate"] <= "2019-12-31")]
J17_2020_df = J17_Well_Data_df.loc[(J17_Well_Data_df["DailyHighDate"] >= "2020-01-01") & (J17_Well_Data_df["DailyHighDate"] <= "2020-12-31")]
J17_2021_df = J17_Well_Data_df.loc[(J17_Well_Data_df["DailyHighDate"] >= "2021-01-01") & (J17_Well_Data_df["DailyHighDate"] <= "2021-12-31")]
J17_2022_df = J17_Well_Data_df.loc[(J17_Well_Data_df["DailyHighDate"] >= "2022-01-01") & (J17_Well_Data_df["DailyHighDate"] <= "2022-07-05")]


In [16]:
Drought_Data_2016_df = Drought_Data_df.loc[(Drought_Data_df["ValidStart"] >= "2016-07-05") & (Drought_Data_df["ValidEnd"] <= "2016-12-31")]
Drought_Data_2017_df = Drought_Data_df.loc[(Drought_Data_df["ValidStart"] >= "2017-01-01") & (Drought_Data_df["ValidEnd"] <= "2017-12-31")]
Drought_Data_2018_df = Drought_Data_df.loc[(Drought_Data_df["ValidStart"] >= "2018-01-01") & (Drought_Data_df["ValidEnd"] <= "2018-12-31")]
Drought_Data_2019_df = Drought_Data_df.loc[(Drought_Data_df["ValidStart"] >= "2019-01-01") & (Drought_Data_df["ValidEnd"] <= "2019-12-31")]
Drought_Data_2020_df = Drought_Data_df.loc[(Drought_Data_df["ValidStart"] >= "2020-01-01") & (Drought_Data_df["ValidEnd"] <= "2020-12-31")]
Drought_Data_2021_df = Drought_Data_df.loc[(Drought_Data_df["ValidStart"] >= "2021-01-01") & (Drought_Data_df["ValidEnd"] <= "2021-12-31")]
Drought_Data_2022_df = Drought_Data_df.loc[(Drought_Data_df["ValidStart"] >= "2022-01-01") & (Drought_Data_df["ValidEnd"] <= "2022-12-31")]

In [17]:
Weather_Data_2016_df

,Temp Max,Temp Avg,Temp Min,Humidity Max,Humidity Avg,Humidity Min,Precipitation,Date
186,98,86.0,79,90,70.3,43,0.0,2016-07-05
187,98,85.2,79,90,70.6,41,0.0,2016-07-06
188,98,85.2,78,90,70.1,42,0.0,2016-07-07
189,98,86.1,79,88,67.0,39,0.0,2016-07-08
190,98,85.5,77,96,70.2,39,0.0,2016-07-09
...,...,...,...,...,...,...,...,...
361,75,67.4,59,96,74.6,48,0.0,2016-12-27
362,84,71.5,63,93,79.7,49,0.0,2016-12-28
363,70,58.7,45,84,37.5,18,0.0,2016-12-29
364,56,51.8,48,49,34.6,25,0.0,2016-12-30


In [22]:
J17_2022_df

,Site,DailyHighDate,WaterLevelElevation,Restriction Level
0,J17WL,2022-07-05,635.98,restriction 3
1,J17WL,2022-07-04,636.85,restriction 3
2,J17WL,2022-07-03,637.15,restriction 3
3,J17WL,2022-07-02,636.71,restriction 3
4,J17WL,2022-07-01,636.55,restriction 3
...,...,...,...,...
181,J17WL,2022-01-05,662.89,no restriction
182,J17WL,2022-01-04,663.02,no restriction
183,J17WL,2022-01-03,663.25,no restriction
184,J17WL,2022-01-02,663.63,no restriction


In [16]:
#Drought_Data_df

# Groupby Month and do the avg. for each month which will be utilized for daily avg. value

In [ ]:
#After we have cleaned data we will do the 10- day bucket sort to compare water days restrictons(SAWS uses a 10 day)

#we are looking for the date and a "drought level" (D01, D02, D03, D04) we will use both J17_well_Data_df and Drought_Data_df

#use .loc 

In [ ]:
#based on new data, start analyzing if the following day had higher vs lower vs equal compared to the restriction day.
#It will be a For loop thru previous cell using the +1



#Final data will be used to creat plots

In [ ]:
#
#Plot a bar chart using Temperature vs Water level to show correlation
#explorepossible analysis

In [ ]:
#
#Plot a bar chart using Humidity vs Water level to show correlation
#explorepossible analysis

In [ ]:
#
#Plot a bar chart using Percipitation vs Water level to show correlation
#explorepossible analysis